## **Importing Modules**

In [1]:
import pandas as pd
import numpy as np

## **Reading Data using Pandas**

We use pandas **_read_csv_** function to read the csv file in Python and Pandas DataFrame method to convert file into data frame.

In [2]:
df = pd.DataFrame(pd.read_csv('/content/train (1).csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

**Seperating out the columns which have more than 35% of the values missing in the dataset**

In [4]:
# df.isnull().sum() returns a pandas series with columns name as the label index
# and total count of null values in the columns as it's value
# And we are storing only those columns which have more than 35% of the data missing.
 
drop_col=df.isnull().sum()[df.isnull().sum()>(35/100*df.shape[0])]
drop_col

Cabin    687
dtype: int64

In [5]:
drop_col.index

Index(['Cabin'], dtype='object')

In [6]:
df.drop(drop_col.index, axis=1, inplace=True)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [7]:
df.fillna(df.mean(), inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Because **Embarked** contains string values, we see the details of that column seperately from others as strings does not have mean and all

In [8]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For **Embarked** attribute, we fill the NULL values with the most frequent value in the column.

In [9]:
df['Embarked'].fillna('S',inplace=True)

In [10]:
df.isnull().sum()      # Now all the NULL values have been filled

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [11]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


**sibsp**: Number of Siblings/Spouses abroad
**patch:** Number of Parents/Children abroad
 
So we can make a new column family_size by combining these two columns

In [12]:
df['FamilySize']=df['SibSp']+df['Parch']
df.drop(['SibSp','Parch'], axis=1, inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


**FamilySize in the ship does not have much correlance with survival rate** <br>
Let's check if wether the person was alone or not can affect the survival rate.

In [13]:
df['Alone']=[0 if df['FamilySize'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,FamilySize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [14]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

If the person is alone he/she has less chance of surving.<br><br> 
The reason might be the person who is traveling with his/her family might be belonging to rich class and might be prioritised over other.

In [15]:
df[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


So we can see if the person was not alone, the chance the ticket price is higher is high

In [16]:
df['Sex']=[0 if df['Sex'][i]=='male' else 1 for i in df.index]  # 1 for female, 0 for male
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

It shows, female passengers have more chance of surving than male ones.
<br>
It shows women were prioritised over men.

In [17]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

## **CONCLUSION**

1) Female were prioritised over men.<br> 
2) People with high class or Rich people have higher survival rate than others. The hieratic might have been followed while saving the passengers.<br>
3) Passengers travelling with their family have higher survival rate.<br>
4) Passengers who borded the ship at Cherbourg, survived more in proportion then the others.